In [150]:
import xmltodict as l
from urllib.request import urlopen
import datetime
import pytz
import influxdb

In [134]:
def str_to_nano(timestr):
    epoch0 = datetime.datetime(1970,1,1)
    epoch0 = epoch0.replace(tzinfo = pytz.UTC)
    
    dt = datetime.datetime.strptime(timestr,'%m/%d/%y %H:%M')
    # convert to UTC time
    dt = dt + datetime.timedelta(hours=4)

    return str(int((dt.replace(tzinfo = pytz.UTC) - epoch0).total_seconds()*1000000000))

In [135]:
def format_for_influx(entry,flux_list):
    timestr = entry['@READ_DT']
    time_nano = str_to_nano(timestr)
    
    for site in entry['SITE']:
        name = site['@NAME']
        
        for measure in site:
            if measure =='@NAME':
                pass
            else:
                measurement = site[measure]
                measure_string = measure+',site='+name+' value='+measurement+' '+str_to_nano(timestr)
                flux_list.append(measure_string)

In [148]:
def write_to_infDB(flux_list):
    host = 'data.open-storm.org'
    port = 8086
    username = 'glwa_daemon'
    password = 'AttackStrawPebble'
    database = 'GLWA'
    influx_client = influxdb.InfluxDBClient(host=host, port=port, database=database,username=username, password=password)

    try:
        influx_client.write_points(flux_list,protocol='line')
    except:
        print('Error Writing to InfluxDB')

In [146]:
file = urlopen('http://gdrss.glwater.org/xml/?FileName=UOFM.xml')
data = file.read()
file.close()

data = l.parse(data)

# list of ordered dictionaries
# Each index is a timestep of measurements.
data = data['DATA']['READ_DT']

In [149]:
flux_list = []
for entry in data:
    format_for_influx(entry,flux_list)
write_to_infDB(flux_list)

In [159]:
host = 'data.open-storm.org'
port = 8086
username = 'glwa_daemon'
password = 'AttackStrawPebble'
database = 'GLWA'
influx_client = influxdb.InfluxDBClient(host=host, port=port, database=database,username=username, password=password)

a =influx_client.query("SELECT * FROM LVL")

AttributeError: 'generator' object has no attribute 'keys'